<a href="https://colab.research.google.com/github/AnnSenina/python_hse_2024/blob/main/notebooks/9_%D0%92%D0%B8%D0%B7%D1%83%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%B8_%D0%A2%D0%B5%D1%81%D1%82%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Визуализации_ Категориальные данные

In [2]:
# все import разом

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats.contingency import association

Простые графики можно строить в matplotlib (а еще он отлично интергрирован с pandas), но работать категориальными данными удобнее в seaborn

Seaborn — это более новая надстройка над matplotlib, которая заметно проще и "сразу делает красиво" (но эта работа происходит незаметно для нас, внутри - "под капотом"). Документация [здесь](https://seaborn.pydata.org/)

Часто категориальные данные просто показызывают цветом на графиках, визуализирующих распределение числовых переменных

In [ ]:
tips = sns.load_dataset("tips") # один из стандартных датасетов в seaborn
tips

In [ ]:
sns.lineplot(x = tips.index, y = 'total_bill', data = tips)
sns.lineplot(x = tips.index, y = 'tip', data = tips);

In [ ]:
# можно еще проще:
sns.lineplot(data = tips[['total_bill', 'tip']]);

Давайте визуализируем распределения общего счета по дням недели. Для визуализации потребуется метод .catplot() Заголовки соответвующих ячеек присвоим аргументам x и y.

x для такого типа графики - столбец для группировки.

Тип отображения (параметр kind) тоже можно менять (попробуйте "point", "bar", "strip", "swarm", "box", "violin" или "boxen")

In [ ]:
sns.catplot(x = "day", y = "total_bill", kind= "violin", data = tips);

Можно достаточно легко и быстро отобразить на графике несколько показателей:

Ниже - распределение значений итогового счета (ось y), сгруппированного по полу (ось x). Дополнительно цветом (параметр hue) можно закодировать еще одно значение с помощью цвета. Например,является ли посетитель курильщиком

In [ ]:
sns.catplot(x = "sex", y = "total_bill", hue = "smoker", kind = "bar", data = tips);

In [ ]:
sns.catplot(x = "total_bill", y = "tip", kind = "strip", hue='sex', data=tips);
# что идет не так: x - не категориальный показатель, а числовой
# более удачный вариант этой визуализации - jointplot

In [ ]:
# jointplot - показывает распределение по 2 числовым переменным + в hue можно добавить категориальные данные
sns.jointplot(x ='total_bill', y = 'tip', hue = 'sex', data = tips);

In [ ]:
# можно строить линейную регрессию (для определения зависимости двух показателей, рассчитывается функция линейной зависимости)
# lmplot - модель линейной регресии
sns.lmplot(x ='total_bill', y = 'tip', data = tips);

In [ ]:
# заодно проверим, меняются ли счет / чаевые в связи со временем посещения - в обед или ужин
sns.lmplot(x ='total_bill', y = 'tip', hue = 'time', data = tips);

### Задание

Ниже - загрузка датасета про пингвинов из seaborn (можно найти на гитхабе создателей библиотеки или нашем гитхабе курса)

Постройте:

- catplot
- jointplot
- lmplot
(для любых показателей на ваш выбор)

In [ ]:
peng_data = sns.load_dataset('penguins') # еще один из стандартных датасетов в seaborn
peng_data

In [ ]:
# @title
sns.catplot(x = "species", y = "bill_depth_mm", kind = "strip", hue='island', data=peng_data);

In [ ]:
# @title
sns.jointplot(x = "bill_length_mm", y = "flipper_length_mm", hue = "species", data=peng_data);

In [ ]:
# @title
sns.lmplot(x = "bill_length_mm", y = "flipper_length_mm", hue = "species", data=peng_data);

О регрессии мы еще поговорим в будущем...

Но сейчас мы займемся категориальными данными вплотную :)

На них можно тестировать статистические гипотезы, проверять связь между двумя категориальными переменными и т.д.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/AnnSenina/Other/main/gender_preferences.csv', sep=',')
df

In [ ]:
df_gender_color = pd.crosstab(df['Gender'], df['Favorite Color'])
df_gender_color

# Тесты и проверка гипотез

## критерий хи-квадрат

Придуман Пирсоном - на этом критерии основано множество статистических измерений. Считается фундаментом современной статистики

Критерий проверяет, есть ли соответсвие между двумя распределениями

* критерий согласия Пирсона (проверка, согласуются ли распределения реальных данных и ожидаемых ИЛИ: согласуются ли 2 наблюдаемых распределения) + [документация](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html)
* тест независимости (проверка, являются ли две переменные (не)зависимыми) - самый полезный для нас + [документация](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html)

(есть и многие другие метрики, основанные на хи-квадрат)

**Важно! Как правило, наш главный показатель на сегодня - p-value (значимость, достоверность связи)**

In [ ]:
# Проблема: Пирсон думал о больших числах!
# Если в нашей таблице частот есть значения от 0 до 5 - критерий хи-квадрат не подходит
df_gender_color = df_gender_color[['Cool', 'Warm']]
df_gender_color

In [ ]:
observed = [17,	13]
# пусть я (под властью стереотипов) ожидаю, что 25 из 30 женщин выберет теплые цвета

expected = [5, 25] # ожидания

chisq, p = stats.chisquare(observed, expected)

print('χ² =', round(chisq, 2), 'p-value =',  round(p, 2))

In [ ]:
# популярный вопрос от магистров: в p-value правда 0?
p # нет, в коде выше округление до двух заков после запятой

In [ ]:
# обычно выбирают доверительный интервал 95%
p < 0.05

# H 0 : (нулевая гипотеза) Переменная следует за гипотетическим распределением.
# H 1 : (альтернативная гипотеза) Переменная не подчиняется предполагаемому распределению.

# Поскольку значение p меньше 0,05, мы можем отвергнуть нулевую гипотезу о согласованности распределений,
# и принять альтернативную: одно распределение статистически отличается от другого

# == данные выборки и наши ожидания не согласуются

In [ ]:
observed = [17,	13]
expected = [15, 15] # изменим гипотезу: я ожидала, что женщины одинаково часто выбирают теплые и холодные цвета

chisq, p = stats.chisquare(observed, expected)

print('χ² =', round(chisq, 2), 'p-value =',  round(p, 2))

# что можем сказать о таком ожидании?
# нельзя отвергнуть нулевую гипотезу == мы не доказали, что различия есть
# -> статистически значимых различий в распределениях нет

In [ ]:
# наконец, проверим, есть ли статистически значимые различия при выборе любимого цвета у мужчин и женщин

df_gender_color = pd.crosstab(df['Gender'], df['Favorite Color'])
df_gender_color = df_gender_color[['Cool', 'Warm']]
df_gender_color

In [ ]:
# проблема: к таким данным нельзя посчитать хи-квадрат:
print(df_gender_color.loc['F'].sum())
print(df_gender_color.loc['M'].sum())

In [ ]:
# перейдем к процентам
cold_warm = df[['Gender', 'Favorite Color']]
cold_warm = cold_warm[cold_warm['Favorite Color'] != 'Neutral']
df_gender_color = pd.crosstab(cold_warm['Gender'], cold_warm['Favorite Color'], normalize='index')
df_gender_color

In [ ]:
chisq, p = stats.chisquare(df_gender_color.loc['F'], df_gender_color.loc['M'])[:2]

print('χ² =', round(chisq, 2), 'p-value =',  round(p, 2))

# нельзя отвергнуть нулевую гипотезу == мы не доказали, что различия есть
# -> статистически значимых различий в распределениях нет

Как использовать в вашем исследовании:

вы можете проверить какой-нибудь вывод из научной литературы и реальные данные из какой-то выборки

### тест независимости хи-квадрат

In [ ]:
df_gender_color

In [ ]:
# тест независимости гораздо интереснее для нас - по логике ближе к корреляционному анализу

# H 0 : (нулевая гипотеза) Две переменные независимы.
# H 1 : (альтернативная гипотеза) Две переменные не являются независимыми = т.е. зависимые, совместно распределяются.

chisq, p = stats.chi2_contingency(df_gender_color)[:2]

print('χ² =', round(chisq, 2), 'p-value =',  round(p, 2))

# p-value < 0.05?
# нельзя отвернуть нулевую гипотезу о независимости переменных
# == наши данные не позволяют подтвердить, что переменные зависимы, связаны

# Точный тест Фишера

Когда у нас небольшое количество значений (**таблица 2 на 2**), можно посчитать точный тест Фишера!

Это хорошая альтернатива тесту независимости х-квадрат, считается, что искажения в этом тесте минимальны

[Документация](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.fisher_exact.html)

**Важное отличие**: тест Фишера работает с разреженными выборками (значения частот могут быть менее 5)

In [ ]:
df["Favorite Color"]

In [ ]:
df_cat = pd.get_dummies(df["Favorite Color"]) # превращает категории в бинарные столбцы True / False
df_cat['Gender'] = df['Gender']
df_cat

In [ ]:
pd.crosstab(df_cat['Gender'], df_cat['Neutral'])

In [ ]:
# Точный тест Фишера использует следующие нулевые и альтернативные гипотезы (такие же, как тест независимости хи-квадрат):

# H 0 : (нулевая гипотеза) Две переменные независимы.

# H 1 : (альтернативная гипотеза) Две переменные не являются независимыми.

fisher, p = stats.fisher_exact(pd.crosstab(df_cat['Gender'], df_cat['Warm']))

print('значение теста =', round(fisher, 2), 'p-value =',  round(p, 2))

# Переменные не связаны ИЛИ мы не смогли отвергнуть их независимость (лучше)
# == Выбор нейтрального цвета не зависит от гендера

# Коэффициент Крамера

мера связи двух номинальных переменных на основе критерия хи-квадрат (= почти корреляционный анализ номинальных данных)

[Документация](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.contingency.association.html)

"Корреляция" -> скорее про независимость

In [ ]:
pd.crosstab(df_cat['Gender'], df_cat['Neutral'])

In [ ]:
association(pd.crosstab(df_cat['Gender'], df_cat['Neutral']), method="cramer")
# проблема: нет p-value

Где взять p-value? Из критерия хи-квадрат

Готового кода для того, чтобы вытащить p-value при подсчете коэффициента Крамера, в библиотеках нет

* можно посчитать для этих же данных критерий независимости хи-квадрат
* или найти того, кто уже это прикрутил в своей функции для подсчета коэффициента корреляции Крамера ([отсюда](https://github.com/glebmikha/mrstat/blob/master/mrstat.py))

In [ ]:
def vcramer(table):
    chi, p, _, _ = stats.chi2_contingency(table,correction=False)
    n = table.sum()
    r,c = table.shape
    return np.sqrt(chi/(n*(min(r,c)-1.))), p

vcramer(pd.crosstab(df_cat['Gender'], df_cat['Neutral']).values)

In [ ]:
vcram, p = vcramer(pd.crosstab(df_cat['Gender'], df_cat['Neutral']).values)

print('значение теста =', round(vcram, 2), 'p-value =',  round(p, 2))